### Word Group In Youtube Sentence

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 200  # 28  # native word end index

# sentence ratio and time shift
adjust_sent_word_ratio = 50
adjust_text_word_ratio = 20
shift = 0.3  # sentence time shift

# shared word frequency
shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = True  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word
etymology_word = False  # True for adding etymology word

# adding output file extention
if (not prefix_suffix) & etymology_word & native_word:
    file_ext = "1"
elif (not prefix_suffix) & etymology_word & (not native_word):
    file_ext = "2"
elif prefix_suffix & etymology_word & native_word:
    file_ext = "3"
elif prefix_suffix & etymology_word & (not native_word):
    file_ext = "4"
elif prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
# 1 => for native word and etymology word without prefix suffix. 
# 2 => for only etymology word without prefix suffix. 
# 3 => for native word and etymology word with prefix suffix. prefix_suffix, native_word and etymology_word must be True. 
# 4 => for only etymology word with prefix suffix.
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")


5


In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/2-Word Group In Youtube Sentence/{lang_folder.capitalize()} {lang_pair.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def exract_word_group(text, word_list):
    '''
    exract_word_group(text, word_list): \n
    text is a string sentence, word_list occurs words 
    '''
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []

    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass

    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []

    text_list = []
    for k in max(all_index_list, key=len):  # any error convert k to i
        word = words[k]
        text_list.append(word)
    text = " ".join(text_list)
    
    return text 

In [6]:
def df_exract_word_group(df, source_text_column, opt_column, word_list, sent_len=False, sent_len_num=2):
    '''
    df_exract_word_group(df_adjust_text_ratio, sentence, video_id, word_list, sent_len=False, sent_len_num=2): \n
    df_adjust_text_ratio is a dataframe and it includes sentence and video_id columns. \n
    sentence is a string sentence. video_id is optional column value. word_list occurs words \n
    sent_len is sentence length condition. sent_len_num is sentence occurs how many word at least. 
    '''
    df.reset_index(drop=True, inplace=True)
    df_new = pd.DataFrame()
    for i in range(len(df)):
        source_text = df.loc[i,f"{source_text_column}"]
        opt_var = df.loc[i,f"{opt_column}"]
        words = re.findall(r"\w+", source_text, re.UNICODE)
        
        index_list = []
        for j in range(len(words)):
            if words[j] in word_list:
                index_list.append(j)
            else:
                pass

        all_index_list = []
        var_index_list = []
        for k in range(len(index_list)):
            try:
                var1 = index_list[k] + 1  
                var2 = index_list[k+1]
            except:
                var1 = index_list[k] + 1  
                var2 = index_list[-1]
            if var1 == var2:
                var3 = index_list[k]
                var_index_list.append(var3)
            else:
                var3 = index_list[k]
                var_index_list.append(var3)
                var4 = list(var_index_list)
                all_index_list.append(var4)
                var_index_list = []

        text_all_list = []
        for m in all_index_list:
            text_list = [] 
            for n in m:
                word = words[n]
                text_list.append(word)
                if sent_len:
                    if len(text_list) >= sent_len_num:
                        text = " ".join(text_list)
                        text_all_list.append(text)
                    else:
                        pass
                else:
                   text = " ".join(text_list)
                   text_all_list.append(text) 
            
        for search_string in text_all_list:
            #df_var["index"] = i
            #df_var["search_string"] = search_string
            #df_var["sentence"] = source_text
            #df_var["video_id"] = opt_var
            df_list = []
            df_list.append([i,search_string,source_text,opt_var])
            df_var = pd.DataFrame(df_list,columns=["index","search_string",f"{source_text_column}",f"{opt_column}"])
            df_new = pd.concat([df_new, df_var], axis=0)

    return df_new 

In [7]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [8]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
988210,karneleme,5
988211,karnaya,5
988212,dörtlümüzün,5
988213,karnavalınız,5


In [9]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [10]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
1478,durmam,2931
1479,almakla,2902
1480,durmalısın,2898
1481,almaları,2869


In [11]:
df_pair = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.lower().capitalize()}/{lang_folder.capitalize()}_{lang_pair.lower().capitalize()}_Shared_Vocabulary.xlsx")
#df_pair = df_pair.head()
df_pair

,dict_entry_main,english_word
0,abaküs,abacus
1,abluka,blockade
2,absorbe,absorb
3,absürt,absurd
4,açelya,azalea
...,...,...
1773,zebra,zebra
1774,zikzak,zigzag
1775,zombi,zombie
1776,zooloji,zoology


In [12]:
# Option
if prefix_suffix:
    df_prefix_suffix_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Word_Prefix_Suffix_Custom_Result.xlsx")
    df_prefix_suffix_select = df_prefix_suffix_select.loc[:,["search_word","word"]]
    df_prefix_suffix_select.rename(columns={"search_word":"dict_entry_main"}, inplace=True)
    df_pair = pd.merge(df_pair,df_prefix_suffix_select, how="inner", on="dict_entry_main")
    df_pair.drop_duplicates(inplace=True)
    df_pair.reset_index(drop=True, inplace=True)
    df_pair = df_pair.loc[:,["word",f"{lang_pair.lower()}_word"]]
    df_pair.rename(columns={"word":"dict_entry_main"}, inplace=True)
else:
    pass

if etymology_word:
    df_pair
else:
    df_pair = df_pair.head(0)
    
df_pair

,dict_entry_main,english_word


In [13]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [14]:
word_select = df_word["word"].values.tolist()
words = df_pair["dict_entry_main"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)
words_set = set(words)

In [15]:
word_list = list(word_select_set.union(words_set.difference(disable_word_set)))

#### Youtube Sentence Analysis

In [16]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,00:11:07.990,00:11:14.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,00:11:14.560,00:11:21.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,00:11:21.040,00:11:27.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,00:11:27.880,00:11:32.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [17]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [18]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...,...
3063204,3063204,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,3063205,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,3063206,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,3063207,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [19]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [20]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id","word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI,44.5,"kullanılıyor, araçlar, kamyon, özgür, normalde","büyük, bunlar, daha, için"
1,1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI,16.8,"tarafı, arka, kolay, burcu, bağlamak",olmayacak
2,2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI,0.1,"attıracağım, arabaya, tur, yarım, özgür",
3,3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,66.8,burcu,"devam, et, tamam, oldu, biraz, daha"
4,4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI,33.4,"tekere, artarsa, takacağız, kalınlığı, kar, burcu","ama, iki, şimdilik"
...,...,...,...,...,...,...,...,...
3063204,3063204,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA,9.2,"önerilen, kartlara, açıklamalar, oynatma, link...",ve
3063205,3063205,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA,18.3,"dilediğiniz, kenar, videoda, yapabilirsiniz, e...","bu, bunlardan"
3063206,3063206,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA,26.8,"elle, videoda, farklı, youtube, görüşene, ücre...","lütfen, bir, sonraki, olan"
3063207,3063207,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA,45.6,"beğenmeyi, unutmayın, abone, kanalıma, videomu...","olarak, çok, iyi, kendinize, bakın"


In [21]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI,44.5,"kullanılıyor, araçlar, kamyon, özgür, normalde","büyük, bunlar, daha, için"
1,1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI,16.8,"tarafı, arka, kolay, burcu, bağlamak",olmayacak
2,2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI,0.1,"attıracağım, arabaya, tur, yarım, özgür",NaN
3,3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,66.8,burcu,"devam, et, tamam, oldu, biraz, daha"
4,4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI,33.4,"tekere, artarsa, takacağız, kalınlığı, kar, burcu","ama, iki, şimdilik"
...,...,...,...,...,...,...,...,...
3063204,3063204,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA,9.2,"önerilen, kartlara, açıklamalar, oynatma, link...",ve
3063205,3063205,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA,18.3,"dilediğiniz, kenar, videoda, yapabilirsiniz, e...","bu, bunlardan"
3063206,3063206,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA,26.8,"elle, videoda, farklı, youtube, görüşene, ücre...","lütfen, bir, sonraki, olan"
3063207,3063207,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA,45.6,"beğenmeyi, unutmayın, abone, kanalıma, videomu...","olarak, çok, iyi, kendinize, bakın"


In [22]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result["word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,66.8,burcu,"devam, et, tamam, oldu, biraz, daha"
1,8,248.832,252.736,çünkü durum şu durumu göstereyim size,8V9tq1pe8eI,50.1,"durum, göstereyim, durumu","şu, çünkü, size"
2,9,254.325,256.668,bu şekilde bir dakika özgür,8V9tq1pe8eI,80.1,özgür,"bir, dakika, bu, şekilde"
3,10,258.199,260.573,kaldırabilecek miyim evet,8V9tq1pe8eI,66.8,kaldırabilecek,"miyim, evet"
4,14,308.735,312.000,özgür nehircim bana yardım eder misin,8V9tq1pe8eI,50.1,"eder, nehircim, özgür","misin, bana, yardım"
...,...,...,...,...,...,...,...,...
1375273,3063187,464.480,472.820,üzerine alıyorum ilmeği mi çıkarttıktan sonra ...,MvWp9pWLihA,57.2,"çıkarttıktan, ilmeği, üzerine","önce, alıyorum, mi, sonra"
1375274,3063191,503.190,513.210,bu birinci ve hemen altındaki iki ve hemen yan...,MvWp9pWLihA,50.1,"yanındaki, altındaki, birinci","bu, iki, ve, hemen, üç"
1375275,3063192,543.360,550.110,aslında modelimizin kurulumunu yapmış olduk şu...,MvWp9pWLihA,50.1,"kurulumunu, modelimizin, yapmış","şuraya, olduk, aslında"
1375276,3063198,627.450,636.000,devam edeceğiz,MvWp9pWLihA,50.1,edeceğiz,devam


In [23]:
df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_sentence_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word,search_string
0,3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,66.8,burcu,"devam, et, tamam, oldu, biraz, daha",biraz daha devam et devam et tamam oldu
1,8,248.832,252.736,çünkü durum şu durumu göstereyim size,8V9tq1pe8eI,50.1,"durum, göstereyim, durumu","şu, çünkü, size",çünkü
2,9,254.325,256.668,bu şekilde bir dakika özgür,8V9tq1pe8eI,80.1,özgür,"bir, dakika, bu, şekilde",bu şekilde bir dakika
3,10,258.199,260.573,kaldırabilecek miyim evet,8V9tq1pe8eI,66.8,kaldırabilecek,"miyim, evet",miyim evet
4,14,308.735,312.000,özgür nehircim bana yardım eder misin,8V9tq1pe8eI,50.1,"eder, nehircim, özgür","misin, bana, yardım",bana yardım
...,...,...,...,...,...,...,...,...,...
1375273,3063187,464.480,472.820,üzerine alıyorum ilmeği mi çıkarttıktan sonra ...,MvWp9pWLihA,57.2,"çıkarttıktan, ilmeği, üzerine","önce, alıyorum, mi, sonra",sonra önce
1375274,3063191,503.190,513.210,bu birinci ve hemen altındaki iki ve hemen yan...,MvWp9pWLihA,50.1,"yanındaki, altındaki, birinci","bu, iki, ve, hemen, üç",iki ve hemen
1375275,3063192,543.360,550.110,aslında modelimizin kurulumunu yapmış olduk şu...,MvWp9pWLihA,50.1,"kurulumunu, modelimizin, yapmış","şuraya, olduk, aslında",olduk şuraya
1375276,3063198,627.450,636.000,devam edeceğiz,MvWp9pWLihA,50.1,edeceğiz,devam,devam


In [25]:
df_word_group_time_loc_in_sent_result = word_group_time_loc(df_adjust_sentence_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,biraz daha devam et devam et tamam oldu,197.690889,201.338000,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
1,çünkü,248.832000,249.465081,çünkü durum şu durumu göstereyim size,8V9tq1pe8eI
2,bu şekilde bir dakika,254.325000,256.234111,bu şekilde bir dakika özgür,8V9tq1pe8eI
3,miyim evet,259.528440,260.573000,kaldırabilecek miyim evet,8V9tq1pe8eI
4,bana yardım,309.970405,311.117568,özgür nehircim bana yardım eder misin,8V9tq1pe8eI
...,...,...,...,...,...
1375273,sonra önce,470.985200,472.820000,üzerine alıyorum ilmeği mi çıkarttıktan sonra ...,MvWp9pWLihA
1375274,iki ve hemen,508.473273,511.023818,bu birinci ve hemen altındaki iki ve hemen yan...,MvWp9pWLihA
1375275,olduk şuraya,548.355000,550.110000,aslında modelimizin kurulumunu yapmış olduk şu...,MvWp9pWLihA
1375276,devam,627.450000,631.114286,devam edeceğiz,MvWp9pWLihA


In [26]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,biraz daha devam et devam et tamam oldu,197.390889,201.638000,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
1,çünkü,248.532000,249.765081,çünkü durum şu durumu göstereyim size,8V9tq1pe8eI
2,bu şekilde bir dakika,254.025000,256.534111,bu şekilde bir dakika özgür,8V9tq1pe8eI
3,miyim evet,259.228440,260.873000,kaldırabilecek miyim evet,8V9tq1pe8eI
4,bana yardım,309.670405,311.417568,özgür nehircim bana yardım eder misin,8V9tq1pe8eI
...,...,...,...,...,...
1375273,sonra önce,470.685200,473.120000,üzerine alıyorum ilmeği mi çıkarttıktan sonra ...,MvWp9pWLihA
1375274,iki ve hemen,508.173273,511.323818,bu birinci ve hemen altındaki iki ve hemen yan...,MvWp9pWLihA
1375275,olduk şuraya,548.055000,550.410000,aslında modelimizin kurulumunu yapmış olduk şu...,MvWp9pWLihA
1375276,devam,627.150000,631.414286,devam edeceğiz,MvWp9pWLihA


In [27]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,biraz daha devam et devam et tamam oldu,197,202,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
1,çünkü,249,250,çünkü durum şu durumu göstereyim size,8V9tq1pe8eI
2,bu şekilde bir dakika,254,257,bu şekilde bir dakika özgür,8V9tq1pe8eI
3,miyim evet,259,261,kaldırabilecek miyim evet,8V9tq1pe8eI
4,bana yardım,310,311,özgür nehircim bana yardım eder misin,8V9tq1pe8eI
...,...,...,...,...,...
1375273,sonra önce,471,473,üzerine alıyorum ilmeği mi çıkarttıktan sonra ...,MvWp9pWLihA
1375274,iki ve hemen,508,511,bu birinci ve hemen altındaki iki ve hemen yan...,MvWp9pWLihA
1375275,olduk şuraya,548,550,aslında modelimizin kurulumunu yapmış olduk şu...,MvWp9pWLihA
1375276,devam,627,631,devam edeceğiz,MvWp9pWLihA


In [28]:
df_word_group_time_loc_in_sent_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_sent_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,nasılsınız teşekkür ederim siz nasılsınız bend...,109,115,nasılsınız teşekkür ederim siz nasılsınız bend...,_Tr90HKsUQ4
1,bak var böyle bir şey var mı öyle bir şey olsa...,949,954,sürü bak var böyle bir şey var mı öyle bir şey...,XswBqDSbWjI
2,var gelirse gelir ve ne zaman gelir ve ne zama...,3416,3423,bu ilgi var gelirse gelir ve ne zaman gelir ve...,8LQvYszWb6k
3,için önemli olan şey şu sizin şimdi benim için...,3164,3169,için önemli olan şey şu sizin şimdi benim için...,Rm6zwwnAVeA
4,teşekkür ederim iyiyim siz nasılsınız teşekkür...,96,107,teşekkür ederim iyiyim siz nasılsınız teşekkür...,DEiWkLwHS6Y
...,...,...,...,...,...
1375273,o,1276,1276,o imkansız,pTTVH1RZDp0
1375274,o,192,194,o,n_GeXW7RWrY
1375275,o,2186,2187,o kimmiş,8l88wzheGsc
1375276,o,2185,2186,o kimmiş ya,8l88wzheGsc


In [29]:
df_word_group_time_loc_in_sent_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_in_sent_result['video_id'].map(str)+"&t="+df_word_group_time_loc_in_sent_result['start_time'].map(str)+"s"
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,nasılsınız teşekkür ederim siz nasılsınız bend...,109,115,nasılsınız teşekkür ederim siz nasılsınız bend...,_Tr90HKsUQ4,https://www.youtube.com/watch?v=_Tr90HKsUQ4&t=...
1,bak var böyle bir şey var mı öyle bir şey olsa...,949,954,sürü bak var böyle bir şey var mı öyle bir şey...,XswBqDSbWjI,https://www.youtube.com/watch?v=XswBqDSbWjI&t=...
2,var gelirse gelir ve ne zaman gelir ve ne zama...,3416,3423,bu ilgi var gelirse gelir ve ne zaman gelir ve...,8LQvYszWb6k,https://www.youtube.com/watch?v=8LQvYszWb6k&t=...
3,için önemli olan şey şu sizin şimdi benim için...,3164,3169,için önemli olan şey şu sizin şimdi benim için...,Rm6zwwnAVeA,https://www.youtube.com/watch?v=Rm6zwwnAVeA&t=...
4,teşekkür ederim iyiyim siz nasılsınız teşekkür...,96,107,teşekkür ederim iyiyim siz nasılsınız teşekkür...,DEiWkLwHS6Y,https://www.youtube.com/watch?v=DEiWkLwHS6Y&t=96s
...,...,...,...,...,...,...
1375273,o,1276,1276,o imkansız,pTTVH1RZDp0,https://www.youtube.com/watch?v=pTTVH1RZDp0&t=...
1375274,o,192,194,o,n_GeXW7RWrY,https://www.youtube.com/watch?v=n_GeXW7RWrY&t=...
1375275,o,2186,2187,o kimmiş,8l88wzheGsc,https://www.youtube.com/watch?v=8l88wzheGsc&t=...
1375276,o,2185,2186,o kimmiş ya,8l88wzheGsc,https://www.youtube.com/watch?v=8l88wzheGsc&t=...


In [30]:
df_word_group_time_loc_in_sent_result = df_word_group_time_loc_in_sent_result.head(1000000)

In [31]:
df_word_group_time_loc_in_sent_result.to_excel(f"Word_Group_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

#### Youtube Videoid Text Analysis

In [52]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,00:11:07.990,00:11:14.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,00:11:14.560,00:11:21.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,00:11:21.040,00:11:27.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,00:11:27.880,00:11:32.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [53]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [54]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [55]:
df_videoid_sentence = df_youtube_sentence.groupby("video_id")["sentence"].apply(" ".join).reset_index()
df_videoid_sentence.reset_index(inplace=True)
df_videoid_sentence

,index,video_id,sentence
0,0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...
1,1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...
2,2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...
3,3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...
4,4,00fr8yyBQYo,bill gates and george soros developed a new co...
...,...,...,...
18361,18361,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...
18362,18362,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...
18363,18363,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...
18364,18364,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...


In [56]:
df_videoid_start_time = df_youtube_sentence.groupby("video_id")[["start_time"]].min()
df_videoid_start_time.reset_index(inplace=True)
df_videoid_start_time

,video_id,start_time
0,007uL_OBtPE,6.680
1,00Dl6tNNlZ8,36.803
2,00QJHZSg0UE,0.440
3,00b3ov_NFSI,0.000
4,00fr8yyBQYo,18.000
...,...,...
18361,zzOL9mRRU0c,9.333
18362,zzWQXUKr9OA,1.896
18363,zzXTIIo7EvM,36.932
18364,zzZfnY7Zjf0,0.000


In [57]:
df_videoid_end_time = df_youtube_sentence.groupby("video_id")[["end_time"]].max()
df_videoid_end_time.reset_index(inplace=True)
df_videoid_end_time

,video_id,end_time
0,007uL_OBtPE,136.640
1,00Dl6tNNlZ8,813.903
2,00QJHZSg0UE,27.460
3,00b3ov_NFSI,42.400
4,00fr8yyBQYo,413.000
...,...,...
18361,zzOL9mRRU0c,533.254
18362,zzWQXUKr9OA,27.969
18363,zzXTIIo7EvM,1591.680
18364,zzZfnY7Zjf0,15.000


In [58]:
df_merge_start_end_time = pd.merge(df_videoid_start_time, df_videoid_end_time, how="inner", on="video_id")
df_merge_start_end_time.drop_duplicates(inplace=True)
df_merge_start_end_time

,video_id,start_time,end_time
0,007uL_OBtPE,6.680,136.640
1,00Dl6tNNlZ8,36.803,813.903
2,00QJHZSg0UE,0.440,27.460
3,00b3ov_NFSI,0.000,42.400
4,00fr8yyBQYo,18.000,413.000
...,...,...,...
18361,zzOL9mRRU0c,9.333,533.254
18362,zzWQXUKr9OA,1.896,27.969
18363,zzXTIIo7EvM,36.932,1591.680
18364,zzZfnY7Zjf0,0.000,15.000


In [59]:
df_videoid_sentence = pd.merge(df_videoid_sentence, df_merge_start_end_time,how="inner", on="video_id")
df_videoid_sentence.drop_duplicates(inplace=True)
df_videoid_sentence

,index,video_id,sentence,start_time,end_time
0,0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,6.680,136.640
1,1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...,36.803,813.903
2,2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,0.440,27.460
3,3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,0.000,42.400
4,4,00fr8yyBQYo,bill gates and george soros developed a new co...,18.000,413.000
...,...,...,...,...,...
18361,18361,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,9.333,533.254
18362,18362,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,1.896,27.969
18363,18363,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...,36.932,1591.680
18364,18364,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,0.000,15.000


In [60]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list2 = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_videoid_sentence)))

def videoid_word_ratio(index_num):
    index = df_videoid_sentence.loc[index_num,"index"]
    videoid = df_videoid_sentence.loc[index_num,"video_id"]
    sentence = df_videoid_sentence.loc[index_num,"sentence"]
    

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list2.append([index,videoid,sentence,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(videoid_word_ratio, index_num)

In [61]:
df_text_ratio_result = pd.DataFrame(list(result_list2), columns=["index","video_id","sentence","word_ratio","different_word","intersect_word"])
df_text_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_text_ratio_result.reset_index(drop=True, inplace=True)
df_text_ratio_result.drop(["index"], axis=1, inplace=True)
df_text_ratio_result

,video_id,sentence,word_ratio,different_word,intersect_word
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,20.9,"soğuyunca, karadeniz, kapatmıyoruz, sizlere, m...","ister, ama, iki, olsun, zaman, şunu, şimdi, ve..."
1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...,5.9,"tutsak, kararların, ayda, galiba, kuraklık, ba...","hemen, doğru, hayır, biliyor, burada, tam, ist..."
2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,22.3,"ievlat, gereğini, yakışıyorsunuz, selvi, ölürü...","yapıyorsun, de, benim, sen, dur, biraz, ne, be..."
3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,33.8,"durumunuzda, kopamayan, denen, birbirinden, si...","ama, değil, adam, gece, bir, peki, herkes, asl..."
4,00fr8yyBQYo,bill gates and george soros developed a new co...,0.1,"mrna, twilight, book, mologic, messengerrna, w...",
...,...,...,...,...,...
18361,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,6.9,"birbirinden, yi, besle, galiba, sistemini, arm...","ama, hadi, haydi, diye, olsun, doğru, bir, teş..."
18362,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,27.4,"baştan, birisi, uzaklaş, cinayetler, varsa, dı...","de, en, gibi, o, yoksa, olan, da, bu, seni, sa..."
18363,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...,4.5,"havuz, koridorun, diyebileceğimiz, birbirinden...","hemen, hiç, gerek, harika, doğru, yüzden, bura..."
18364,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,23.4,"hatta, senaryoyu, ihtiyacım, memnunum, setteyk...","ve, için, çok, vardı, kadar, bir, bunun, büyük..."


In [62]:
df_text_ratio_result["different_word"] = df_text_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result["intersect_word"] = df_text_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result 

,video_id,sentence,word_ratio,different_word,intersect_word
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,20.9,"soğuyunca, karadeniz, kapatmıyoruz, sizlere, m...","ister, ama, iki, olsun, zaman, şunu, şimdi, ve..."
1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...,5.9,"tutsak, kararların, ayda, galiba, kuraklık, ba...","hemen, doğru, hayır, biliyor, burada, tam, ist..."
2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,22.3,"ievlat, gereğini, yakışıyorsunuz, selvi, ölürü...","yapıyorsun, de, benim, sen, dur, biraz, ne, be..."
3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,33.8,"durumunuzda, kopamayan, denen, birbirinden, si...","ama, değil, adam, gece, bir, peki, herkes, asl..."
4,00fr8yyBQYo,bill gates and george soros developed a new co...,0.1,"mrna, twilight, book, mologic, messengerrna, w...",NaN
...,...,...,...,...,...
18361,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,6.9,"birbirinden, yi, besle, galiba, sistemini, arm...","ama, hadi, haydi, diye, olsun, doğru, bir, teş..."
18362,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,27.4,"baştan, birisi, uzaklaş, cinayetler, varsa, dı...","de, en, gibi, o, yoksa, olan, da, bu, seni, sa..."
18363,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...,4.5,"havuz, koridorun, diyebileceğimiz, birbirinden...","hemen, hiç, gerek, harika, doğru, yüzden, bura..."
18364,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,23.4,"hatta, senaryoyu, ihtiyacım, memnunum, setteyk...","ve, için, çok, vardı, kadar, bir, bunun, büyük..."


In [63]:
df_adjust_text_ratio = df_text_ratio_result[df_text_ratio_result["word_ratio"] >= adjust_text_word_ratio]
df_adjust_text_ratio.reset_index(inplace=True, drop=True)
df_adjust_text_ratio

,video_id,sentence,word_ratio,different_word,intersect_word
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,20.9,"soğuyunca, karadeniz, kapatmıyoruz, sizlere, m...","ister, ama, iki, olsun, zaman, şunu, şimdi, ve..."
1,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,22.3,"ievlat, gereğini, yakışıyorsunuz, selvi, ölürü...","yapıyorsun, de, benim, sen, dur, biraz, ne, be..."
2,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,33.8,"durumunuzda, kopamayan, denen, birbirinden, si...","ama, değil, adam, gece, bir, peki, herkes, asl..."
3,00pHXwzua50,i hlas suresi meali rahman ve rahim olan allah...,20.1,"suresi, allahın, rahman, rahim, meali, adıyla,...","olan, ve"
4,04cQYx5m3y4,al bakalım yemeyecek misin hadi olmaz ama çok ...,22.0,"dodo, ablasıyla, hayalimdeki, yi, nşallah, kan...","ama, değil, hiç, hadi, bizi, biliyorum, zaman,..."
...,...,...,...,...,...
3864,zwSrvLwBGeM,farkındalıklı bir güne nasıl başlayabilirim en...,22.3,"endişe, başlayabilirim, kaygı, azaltan, farkın...","bir, nasıl"
3865,zwWS6yifThY,kızım kahve istiyorum ben tamam youtube kanalı...,23.0,"olursanız, afiyet, abone, oluruz, olmuş, kızım...","tamam, güzel, olsun, istiyorum, çok, ben, teşe..."
3866,zzNekJQxa3s,merhaba arkadaşlar ben özkan sözbilen bu vlog ...,22.6,"sözbilen, olup, paylaşmayı, yi, yorum, arkadaş...","ve, nasıl, bütün, daha, merhaba, için, çok, bu..."
3867,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,27.4,"baştan, birisi, uzaklaş, cinayetler, varsa, dı...","de, en, gibi, o, yoksa, olan, da, bu, seni, sa..."


In [64]:
#df_adjust_text_ratio.to_excel("Temp.xlsx", index=False)

In [65]:
df_adjust_text_ratio.loc[:,"search_string"] = df_adjust_text_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_text_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,video_id,sentence,word_ratio,different_word,intersect_word,search_string
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,20.9,"soğuyunca, karadeniz, kapatmıyoruz, sizlere, m...","ister, ama, iki, olsun, zaman, şunu, şimdi, ve...",ve şimdi şimdi
1,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,22.3,"ievlat, gereğini, yakışıyorsunuz, selvi, ölürü...","yapıyorsun, de, benim, sen, dur, biraz, ne, be...",ne yapıyorsun
2,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,33.8,"durumunuzda, kopamayan, denen, birbirinden, si...","ama, değil, adam, gece, bir, peki, herkes, asl...",onu ama ne kadar
3,00pHXwzua50,i hlas suresi meali rahman ve rahim olan allah...,20.1,"suresi, allahın, rahman, rahim, meali, adıyla,...","olan, ve",ve
4,04cQYx5m3y4,al bakalım yemeyecek misin hadi olmaz ama çok ...,22.0,"dodo, ablasıyla, hayalimdeki, yi, nşallah, kan...","ama, değil, hiç, hadi, bizi, biliyorum, zaman,...",nasıl ya nasıl ya ne zaman sen
...,...,...,...,...,...,...
3864,zwSrvLwBGeM,farkındalıklı bir güne nasıl başlayabilirim en...,22.3,"endişe, başlayabilirim, kaygı, azaltan, farkın...","bir, nasıl",bir
3865,zwWS6yifThY,kızım kahve istiyorum ben tamam youtube kanalı...,23.0,"olursanız, afiyet, abone, oluruz, olmuş, kızım...","tamam, güzel, olsun, istiyorum, çok, ben, teşe...",istiyorum ben tamam
3866,zzNekJQxa3s,merhaba arkadaşlar ben özkan sözbilen bu vlog ...,22.6,"sözbilen, olup, paylaşmayı, yi, yorum, arkadaş...","ve, nasıl, bütün, daha, merhaba, için, çok, bu...",ve çok daha
3867,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,27.4,"baştan, birisi, uzaklaş, cinayetler, varsa, dı...","de, en, gibi, o, yoksa, olan, da, bu, seni, sa...",ya bu


In [66]:
df_adjust_text_ratio = df_adjust_text_ratio[["search_string","video_id","sentence","word_ratio"]]
df_adjust_text_ratio

,search_string,video_id,sentence,word_ratio
0,ve şimdi şimdi,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,20.9
1,ne yapıyorsun,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,22.3
2,onu ama ne kadar,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,33.8
3,ve,00pHXwzua50,i hlas suresi meali rahman ve rahim olan allah...,20.1
4,nasıl ya nasıl ya ne zaman sen,04cQYx5m3y4,al bakalım yemeyecek misin hadi olmaz ama çok ...,22.0
...,...,...,...,...
3864,bir,zwSrvLwBGeM,farkındalıklı bir güne nasıl başlayabilirim en...,22.3
3865,istiyorum ben tamam,zwWS6yifThY,kızım kahve istiyorum ben tamam youtube kanalı...,23.0
3866,ve çok daha,zzNekJQxa3s,merhaba arkadaşlar ben özkan sözbilen bu vlog ...,22.6
3867,ya bu,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,27.4


In [67]:
df_videoid_sentence = df_videoid_sentence[["video_id","start_time","end_time"]]

In [68]:
df_adjust_text_ratio = pd.merge(df_adjust_text_ratio, df_videoid_sentence, how="inner", on="video_id")
df_adjust_text_ratio

,search_string,video_id,sentence,word_ratio,start_time,end_time
0,ve şimdi şimdi,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,20.9,6.680,136.640
1,ne yapıyorsun,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,22.3,0.440,27.460
2,onu ama ne kadar,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,33.8,0.000,42.400
3,ve,00pHXwzua50,i hlas suresi meali rahman ve rahim olan allah...,20.1,0.000,13.701
4,nasıl ya nasıl ya ne zaman sen,04cQYx5m3y4,al bakalım yemeyecek misin hadi olmaz ama çok ...,22.0,0.350,394.702
...,...,...,...,...,...,...
3864,bir,zwSrvLwBGeM,farkındalıklı bir güne nasıl başlayabilirim en...,22.3,0.000,2.000
3865,istiyorum ben tamam,zwWS6yifThY,kızım kahve istiyorum ben tamam youtube kanalı...,23.0,0.501,155.008
3866,ve çok daha,zzNekJQxa3s,merhaba arkadaşlar ben özkan sözbilen bu vlog ...,22.6,0.000,23.600
3867,ya bu,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,27.4,1.896,27.969


In [69]:
df_word_group_time_loc_in_text_result = word_group_time_loc(df_adjust_text_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_text_result

,search_string,start_time,end_time,sentence,video_id
0,ve şimdi şimdi,60.786645,62.451465,merhaba arkadaşlar bugün sizlere karadeniz yör...,007uL_OBtPE
1,ne yapıyorsun,16.488242,17.716424,ioğluyla biraz vakit geçirsini ievlat bui idün...,00QJHZSg0UE
2,onu ama ne kadar,5.004147,6.221372,evleniyorsun kocam diyorsun aynı yastığa baş k...,00b3ov_NFSI
3,ve,6.476836,7.473273,i hlas suresi meali rahman ve rahim olan allah...,00pHXwzua50
4,nasıl ya nasıl ya ne zaman sen,96.857135,104.055803,al bakalım yemeyecek misin hadi olmaz ama çok ...,04cQYx5m3y4
...,...,...,...,...,...
3864,bir,0.351351,0.486486,farkındalıklı bir güne nasıl başlayabilirim en...,zwSrvLwBGeM
3865,istiyorum ben tamam,7.466480,20.764213,kızım kahve istiyorum ben tamam youtube kanalı...,zwWS6yifThY
3866,ve çok daha,11.225468,12.374532,merhaba arkadaşlar ben özkan sözbilen bu vlog ...,zzNekJQxa3s
3867,ya bu,3.403110,3.930598,bak baştan anlaşalım ya bu cinayete de diğer c...,zzWQXUKr9OA


In [70]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_text_result

,search_string,start_time,end_time,sentence,video_id
0,ve şimdi şimdi,60.486645,62.751465,merhaba arkadaşlar bugün sizlere karadeniz yör...,007uL_OBtPE
1,ne yapıyorsun,16.188242,18.016424,ioğluyla biraz vakit geçirsini ievlat bui idün...,00QJHZSg0UE
2,onu ama ne kadar,4.704147,6.521372,evleniyorsun kocam diyorsun aynı yastığa baş k...,00b3ov_NFSI
3,ve,6.176836,7.773273,i hlas suresi meali rahman ve rahim olan allah...,00pHXwzua50
4,nasıl ya nasıl ya ne zaman sen,96.557135,104.355803,al bakalım yemeyecek misin hadi olmaz ama çok ...,04cQYx5m3y4
...,...,...,...,...,...
3864,bir,0.051351,0.786486,farkındalıklı bir güne nasıl başlayabilirim en...,zwSrvLwBGeM
3865,istiyorum ben tamam,7.166480,21.064213,kızım kahve istiyorum ben tamam youtube kanalı...,zwWS6yifThY
3866,ve çok daha,10.925468,12.674532,merhaba arkadaşlar ben özkan sözbilen bu vlog ...,zzNekJQxa3s
3867,ya bu,3.103110,4.230598,bak baştan anlaşalım ya bu cinayete de diğer c...,zzWQXUKr9OA


In [71]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result

,search_string,start_time,end_time,sentence,video_id
0,ve şimdi şimdi,60,63,merhaba arkadaşlar bugün sizlere karadeniz yör...,007uL_OBtPE
1,ne yapıyorsun,16,18,ioğluyla biraz vakit geçirsini ievlat bui idün...,00QJHZSg0UE
2,onu ama ne kadar,5,7,evleniyorsun kocam diyorsun aynı yastığa baş k...,00b3ov_NFSI
3,ve,6,8,i hlas suresi meali rahman ve rahim olan allah...,00pHXwzua50
4,nasıl ya nasıl ya ne zaman sen,97,104,al bakalım yemeyecek misin hadi olmaz ama çok ...,04cQYx5m3y4
...,...,...,...,...,...
3864,bir,0,1,farkındalıklı bir güne nasıl başlayabilirim en...,zwSrvLwBGeM
3865,istiyorum ben tamam,7,21,kızım kahve istiyorum ben tamam youtube kanalı...,zwWS6yifThY
3866,ve çok daha,11,13,merhaba arkadaşlar ben özkan sözbilen bu vlog ...,zzNekJQxa3s
3867,ya bu,3,4,bak baştan anlaşalım ya bu cinayete de diğer c...,zzWQXUKr9OA


In [72]:
df_word_group_time_loc_in_text_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_text_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_text_result

,search_string,start_time,end_time,sentence,video_id
0,al bakalım hadi gel bakalım bak neler var bura...,171,183,size de zahmet verdik biz ne demek rica ederiz...,iqc5P0IEYTY
1,olur mu öyle olur mu anne teşekkür ederim sen ...,146,159,hadi bakalım yemek hazır toplayın eşyalarınızı...,TjNM6UO7Jus
2,sen iyi misin hayır değilim ben hiç iyi değili...,74,87,hayırdır nereye bu saatte kahveye çıkıyorum ne...,npVFfXBjHbU
3,izin ver lütfen neden ben bu benim senin ve se...,64,82,bu dünyada tek istediğim katliam edy biz orada...,aagvVVNb9F8
4,beni ne istiyorsun ben ne istiyorum sen çok iy...,9,17,benim kızım şu anda gökhanın elinde akgün iyiy...,Ns8xYenvavQ
...,...,...,...,...,...
3864,bu,9,11,i lgili bu akşam saar,Ma16m0VEhuk
3865,de,284,333,video saruhanlı koldere de çekilmiştir,9oZ4FLaC_lk
3866,mi,0,1,mi sti k sukulent dekorasyon,d_LUSNc4zU8
3867,o,0,1,o sizin panelin mezarlıkları için ideal bir bi...,sNOATbKrqnU


In [73]:
df_word_group_time_loc_in_text_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_in_text_result['video_id'].map(str)+"&t="+df_word_group_time_loc_in_text_result['start_time'].map(str)+"s"
df_word_group_time_loc_in_text_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,al bakalım hadi gel bakalım bak neler var bura...,171,183,size de zahmet verdik biz ne demek rica ederiz...,iqc5P0IEYTY,https://www.youtube.com/watch?v=iqc5P0IEYTY&t=...
1,olur mu öyle olur mu anne teşekkür ederim sen ...,146,159,hadi bakalım yemek hazır toplayın eşyalarınızı...,TjNM6UO7Jus,https://www.youtube.com/watch?v=TjNM6UO7Jus&t=...
2,sen iyi misin hayır değilim ben hiç iyi değili...,74,87,hayırdır nereye bu saatte kahveye çıkıyorum ne...,npVFfXBjHbU,https://www.youtube.com/watch?v=npVFfXBjHbU&t=74s
3,izin ver lütfen neden ben bu benim senin ve se...,64,82,bu dünyada tek istediğim katliam edy biz orada...,aagvVVNb9F8,https://www.youtube.com/watch?v=aagvVVNb9F8&t=64s
4,beni ne istiyorsun ben ne istiyorum sen çok iy...,9,17,benim kızım şu anda gökhanın elinde akgün iyiy...,Ns8xYenvavQ,https://www.youtube.com/watch?v=Ns8xYenvavQ&t=9s
...,...,...,...,...,...,...
3864,bu,9,11,i lgili bu akşam saar,Ma16m0VEhuk,https://www.youtube.com/watch?v=Ma16m0VEhuk&t=9s
3865,de,284,333,video saruhanlı koldere de çekilmiştir,9oZ4FLaC_lk,https://www.youtube.com/watch?v=9oZ4FLaC_lk&t=...
3866,mi,0,1,mi sti k sukulent dekorasyon,d_LUSNc4zU8,https://www.youtube.com/watch?v=d_LUSNc4zU8&t=0s
3867,o,0,1,o sizin panelin mezarlıkları için ideal bir bi...,sNOATbKrqnU,https://www.youtube.com/watch?v=sNOATbKrqnU&t=0s


In [74]:
df_word_group_time_loc_in_text_result.to_excel(f"Word_Group_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)

In [76]:
text = df_word_group_time_loc_in_text_result.loc[13,"sentence"]
text

'yarın zeynepin annesine gideceğim çocuğu evlatlık verecekse bazı evrakları toparlamam gerekiyor mahir bey çocuklara bu konuyu unutturun lütfen akıllarına yanlış şeyler sokmayın ne yanlış var bunda çocukların gidecekleri aileleri seçmeleri hakları değil mi kuralları ben koymuyorum ama kuralları yorumlamak senin elinde ne dodoyu eve gönderme önce burada görüşsünler neden görmedin mi kadından hoşlanmadı dokunmak bile istemedi bırak çocuğun burnunu silmedi diye kötülüyorsun kadını git yat ama ya onu üzerlerse sen benim ne zaman kötü bir aileye çocuk verdiğimi gördün i kisi de çocuğun gözlerinin içine baktılar daha ne istiyorsun bana gelenler de öyleydi onlar gibi rol yapmadıklarını nereden biliyorsun ne dedin sen ne dedin ne rolü kim rol yapmış ne diyorsun sen ne rolü beni istiyormuş rolü yapmadılar mı i stediler zaten sana kaza geçirdiler dedik duymadın mı o yüzden gelemediler alamadılar seni yalan söylemene gerek yok ben de istemiyordum onları umurumda bile değil gel buraya gel gel gel 

In [77]:
words = re.findall(r"\w+", text, re.UNICODE)
words

['yarın',
 'zeynepin',
 'annesine',
 'gideceğim',
 'çocuğu',
 'evlatlık',
 'verecekse',
 'bazı',
 'evrakları',
 'toparlamam',
 'gerekiyor',
 'mahir',
 'bey',
 'çocuklara',
 'bu',
 'konuyu',
 'unutturun',
 'lütfen',
 'akıllarına',
 'yanlış',
 'şeyler',
 'sokmayın',
 'ne',
 'yanlış',
 'var',
 'bunda',
 'çocukların',
 'gidecekleri',
 'aileleri',
 'seçmeleri',
 'hakları',
 'değil',
 'mi',
 'kuralları',
 'ben',
 'koymuyorum',
 'ama',
 'kuralları',
 'yorumlamak',
 'senin',
 'elinde',
 'ne',
 'dodoyu',
 'eve',
 'gönderme',
 'önce',
 'burada',
 'görüşsünler',
 'neden',
 'görmedin',
 'mi',
 'kadından',
 'hoşlanmadı',
 'dokunmak',
 'bile',
 'istemedi',
 'bırak',
 'çocuğun',
 'burnunu',
 'silmedi',
 'diye',
 'kötülüyorsun',
 'kadını',
 'git',
 'yat',
 'ama',
 'ya',
 'onu',
 'üzerlerse',
 'sen',
 'benim',
 'ne',
 'zaman',
 'kötü',
 'bir',
 'aileye',
 'çocuk',
 'verdiğimi',
 'gördün',
 'i',
 'kisi',
 'de',
 'çocuğun',
 'gözlerinin',
 'içine',
 'baktılar',
 'daha',
 'ne',
 'istiyorsun',
 'bana',
 'g

In [78]:
index_list = []
for i in range(len(words)):
    if words[i] in word_list:
        index_list.append(i)
    else:
        pass

In [79]:
all_index_list = []
var_index_list = []
for j in range(len(index_list)):
    try:
        var1 = index_list[j] + 1  
        var2 = index_list[j+1]
    except:
        var1 = index_list[j] + 1  
        var2 = index_list[-1]
    if var1 == var2:
        var3 = index_list[j]
        var_index_list.append(var3)
    else:
        var3 = index_list[j]
        var_index_list.append(var3)
        var4 = list(var_index_list)
        all_index_list.append(var4)
        var_index_list = []

In [80]:
all_index_list

[[14],
 [17],
 [20],
 [22],
 [24],
 [31, 32],
 [34],
 [36],
 [39],
 [41],
 [45, 46],
 [48],
 [50],
 [54],
 [60],
 [63],
 [65, 66, 67],
 [69, 70, 71, 72, 73, 74],
 [81],
 [86, 87, 88, 89],
 [91],
 [93, 94],
 [98, 99],
 [101, 102],
 [104],
 [109],
 [111, 112],
 [114],
 [118],
 [121, 122],
 [127, 128, 129],
 [132],
 [135, 136, 137, 138],
 [140],
 [142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153],
 [157],
 [159, 160, 161, 162, 163, 164, 165],
 [168, 169],
 [176, 177],
 [181, 182],
 [185, 186],
 [189],
 [193],
 [196, 197],
 [200, 201],
 [205, 206],
 [208, 209, 210],
 [213, 214, 215],
 [218],
 [220, 221],
 [223, 224, 225],
 [229],
 [231],
 [233, 234],
 [237],
 [243, 244],
 [247, 248],
 [250, 251],
 [254, 255, 256],
 [258],
 [261]]

In [81]:
sorted(all_index_list, key=len, reverse=True)

[[142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153],
 [159, 160, 161, 162, 163, 164, 165],
 [69, 70, 71, 72, 73, 74],
 [86, 87, 88, 89],
 [135, 136, 137, 138],
 [65, 66, 67],
 [127, 128, 129],
 [208, 209, 210],
 [213, 214, 215],
 [223, 224, 225],
 [254, 255, 256],
 [31, 32],
 [45, 46],
 [93, 94],
 [98, 99],
 [101, 102],
 [111, 112],
 [121, 122],
 [168, 169],
 [176, 177],
 [181, 182],
 [185, 186],
 [196, 197],
 [200, 201],
 [205, 206],
 [220, 221],
 [233, 234],
 [243, 244],
 [247, 248],
 [250, 251],
 [14],
 [17],
 [20],
 [22],
 [24],
 [34],
 [36],
 [39],
 [41],
 [48],
 [50],
 [54],
 [60],
 [63],
 [81],
 [91],
 [104],
 [109],
 [114],
 [118],
 [132],
 [140],
 [157],
 [189],
 [193],
 [218],
 [229],
 [231],
 [237],
 [258],
 [261]]

In [82]:
max(all_index_list, key=len)

[142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153]

In [83]:
text_list = []
for i in max(all_index_list, key=len):
    word = words[i]
    text_list.append(word)
text = " ".join(text_list)
text

'bile değil gel buraya gel gel gel buraya gel sen gel buraya'

In [84]:
df_adjust_text_ratio

,search_string,video_id,sentence,word_ratio,start_time,end_time
0,ve şimdi şimdi,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,20.9,6.680,136.640
1,ne yapıyorsun,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,22.3,0.440,27.460
2,onu ama ne kadar,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,33.8,0.000,42.400
3,ve,00pHXwzua50,i hlas suresi meali rahman ve rahim olan allah...,20.1,0.000,13.701
4,nasıl ya nasıl ya ne zaman sen,04cQYx5m3y4,al bakalım yemeyecek misin hadi olmaz ama çok ...,22.0,0.350,394.702
...,...,...,...,...,...,...
3864,bir,zwSrvLwBGeM,farkındalıklı bir güne nasıl başlayabilirim en...,22.3,0.000,2.000
3865,istiyorum ben tamam,zwWS6yifThY,kızım kahve istiyorum ben tamam youtube kanalı...,23.0,0.501,155.008
3866,ve çok daha,zzNekJQxa3s,merhaba arkadaşlar ben özkan sözbilen bu vlog ...,22.6,0.000,23.600
3867,ya bu,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,27.4,1.896,27.969


In [119]:
df_test = df_adjust_text_ratio.iloc[4:5,:]
#df_test.reset_index(drop=True, inplace=True)
df_test

,search_string,video_id,sentence,word_ratio,start_time,end_time
4,nasıl ya nasıl ya ne zaman sen,04cQYx5m3y4,al bakalım yemeyecek misin hadi olmaz ama çok ...,22.0,0.35,394.702


In [133]:
# tested
def df_exract_word_group(df, source_text_column, opt_column, word_list, sent_len=False, sent_len_num=2):
    '''
    df_exract_word_group(df_adjust_text_ratio, sentence, video_id, word_list, sent_len=False, sent_len_num=2): \n
    df_adjust_text_ratio is a dataframe and it includes sentence and video_id columns. \n
    sentence is a string sentence. video_id is optional column value. word_list occurs words \n
    sent_len is sentence length condition. sent_len_num is sentence occurs how many word at least. 
    '''
    df.reset_index(drop=True, inplace=True)
    df_new = pd.DataFrame()
    for i in range(len(df)):
        source_text = df.loc[i,f"{source_text_column}"]
        opt_var = df.loc[i,f"{opt_column}"]
        words = re.findall(r"\w+", source_text, re.UNICODE)
        
        index_list = []
        for j in range(len(words)):
            if words[j] in word_list:
                index_list.append(j)
            else:
                pass

        all_index_list = []
        var_index_list = []
        for k in range(len(index_list)):
            try:
                var1 = index_list[k] + 1  
                var2 = index_list[k+1]
            except:
                var1 = index_list[k] + 1  
                var2 = index_list[-1]
            if var1 == var2:
                var3 = index_list[k]
                var_index_list.append(var3)
            else:
                var3 = index_list[k]
                var_index_list.append(var3)
                var4 = list(var_index_list)
                all_index_list.append(var4)
                var_index_list = []

        text_all_list = []
        for m in all_index_list:
            text_list = [] 
            for n in m:
                word = words[n]
                text_list.append(word)
                if sent_len:
                    if len(text_list) >= sent_len_num:
                        text = " ".join(text_list)
                        text_all_list.append(text)
                    else:
                        pass
                else:
                   text = " ".join(text_list)
                   text_all_list.append(text) 
            
        for search_string in text_all_list:
            #df_var["index"] = i
            #df_var["search_string"] = search_string
            #df_var["sentence"] = source_text
            #df_var["video_id"] = opt_var
            df_list = []
            df_list.append([i,search_string,source_text,opt_var])
            df_var = pd.DataFrame(df_list,columns=["index","search_string",f"{source_text_column}",f"{opt_column}"])
            df_new = pd.concat([df_new, df_var], axis=0)

    return df_new 

In [134]:
df_new = df_exract_word_group(df_adjust_text_ratio, "sentence", "video_id", word_list, sent_len=True, sent_len_num=10)
df_new

,index,search_string,sentence,video_id
0,115,bir anne var tamam ne tamam tamam işte ben de,ne yapıyor mahir uyuyor hiçbir şey yemedi bütü...,1mdeot2yRCc
0,115,bir anne var tamam ne tamam tamam işte ben de ...,ne yapıyor mahir uyuyor hiçbir şey yemedi bütü...,1mdeot2yRCc
0,512,peki böyle bir şeyi senin için benim için beni...,nazım bugün ofise zerrin hanım geldi nehir tes...,8kw9b3KrjGU
0,512,peki böyle bir şeyi senin için benim için beni...,nazım bugün ofise zerrin hanım geldi nehir tes...,8kw9b3KrjGU
0,512,peki böyle bir şeyi senin için benim için beni...,nazım bugün ofise zerrin hanım geldi nehir tes...,8kw9b3KrjGU
...,...,...,...,...
0,3798,bir şey oldu değil mi ne oldu bize hemen söyle,evet o börekçi kadın yüzünden mi acaba ne alak...,ypIZm8Q2WuY
0,3798,bir şey oldu değil mi ne oldu bize hemen söyle...,evet o börekçi kadın yüzünden mi acaba ne alak...,ypIZm8Q2WuY
0,3853,var böyle bir şey gerçek olabilir mi tabii ki ...,hamile kalma yolculuğunda olan anne aday adayl...,zidlx3JojIo
0,3853,var böyle bir şey gerçek olabilir mi tabii ki ...,hamile kalma yolculuğunda olan anne aday adayl...,zidlx3JojIo


In [135]:
df_new.drop_duplicates()

,index,search_string,sentence,video_id
0,115,bir anne var tamam ne tamam tamam işte ben de,ne yapıyor mahir uyuyor hiçbir şey yemedi bütü...,1mdeot2yRCc
0,115,bir anne var tamam ne tamam tamam işte ben de ...,ne yapıyor mahir uyuyor hiçbir şey yemedi bütü...,1mdeot2yRCc
0,512,peki böyle bir şeyi senin için benim için beni...,nazım bugün ofise zerrin hanım geldi nehir tes...,8kw9b3KrjGU
0,512,peki böyle bir şeyi senin için benim için beni...,nazım bugün ofise zerrin hanım geldi nehir tes...,8kw9b3KrjGU
0,512,peki böyle bir şeyi senin için benim için beni...,nazım bugün ofise zerrin hanım geldi nehir tes...,8kw9b3KrjGU
...,...,...,...,...
0,3798,bir şey oldu değil mi ne oldu bize hemen söyle,evet o börekçi kadın yüzünden mi acaba ne alak...,ypIZm8Q2WuY
0,3798,bir şey oldu değil mi ne oldu bize hemen söyle...,evet o börekçi kadın yüzünden mi acaba ne alak...,ypIZm8Q2WuY
0,3853,var böyle bir şey gerçek olabilir mi tabii ki ...,hamile kalma yolculuğunda olan anne aday adayl...,zidlx3JojIo
0,3853,var böyle bir şey gerçek olabilir mi tabii ki ...,hamile kalma yolculuğunda olan anne aday adayl...,zidlx3JojIo


In [ ]:
#text = df_videoid_sentence.loc[0,"sentence"]
#text

In [ ]:
text = "ablan burada nerede i çeriye kadar gelemedi ama benimle buraya kadar geldi sen şimdi git baba sonra beni almaya gel ama hemen gel tamam mı tamam defol defol defol her şeyi mahvettin defol ne işin var burada hayır hayır hayır bırakın bırak beni bırak beni bırak"

In [ ]:
words = re.findall(r"\w+", text, re.UNICODE)
words

In [ ]:
len(words)

In [ ]:
words[1] in word_list

In [ ]:
index_list = []
for i in range(len(words)):
    if words[i] in word_list:
        index_list.append(i)
    else:
        pass

In [ ]:
index_list

In [ ]:
index_list[0] 

In [ ]:
len(index_list)

In [ ]:
all_index_list = []
var_index_list = []
for j in range(len(index_list)):
    try:
        var1 = index_list[j] + 1  
        var2 = index_list[j+1]
    except:
        var1 = index_list[j] + 1  
        var2 = index_list[-1]
    if var1 == var2:
        var3 = index_list[j]
        var_index_list.append(var3)
    else:
        var3 = index_list[j]
        var_index_list.append(var3)
        var4 = list(var_index_list)
        all_index_list.append(var4)
        var_index_list = []

In [ ]:
all_index_list

In [ ]:
all_index_list[0]

In [ ]:
max(all_index_list, key=len)

In [ ]:
text_list = []
for i in max(all_index_list, key=len):
    word = words[i]
    text_list.append(word)
text = " ".join(text_list)
text